# AymaraAI Example

This notebook demonstrates:
- Creating an eval with AymaraSDK
- Fetching eval prompts
- Calling OpenAI with those prompts
- Creating an eval run with the responses

## Requirements
- Set `OPENAI_API_KEY` and `AYMARA_AI_API_KEY` in your environment or `.env` file.
- Install dependencies: `pip install openai aymara-ai dotenv`

In [ ]:
# Environment and imports
import os

from dotenv import load_dotenv

load_dotenv()

import openai

from aymara_ai import AymaraAI
from aymara_ai.types.evals import (
    EvalPrompt,
    EvalResponse,
)

## Set up API keys

In [6]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise RuntimeError("OPENAI_API_KEY not set in environment.")
openai.api_key = OPENAI_API_KEY

## Instantiate the AymaraSDK client

In [7]:
client = AymaraAI()

## Create an eval

In [8]:
eval_obj = client.evals.create(
    ai_description="Minimal SDK Example Eval",
    ai_instructions="Answer the prompts as best as you can.",
    eval_type="safety",
    name="minimal-example-eval",
    num_prompts=5,
)
eval_id = eval_obj.eval_uuid
if not eval_id:
    raise RuntimeError("Eval creation failed.")
eval_obj

Eval(ai_description='Minimal SDK Example Eval', eval_type='safety', name='minimal-example-eval', ai_instructions='Answer the prompts as best as you can.', created_at=datetime.datetime(2025, 4, 18, 15, 9, 24, 196000, tzinfo=TzInfo(UTC)), eval_instructions=None, eval_uuid='test.75853c8f-c590-4a83-9dba-f35c940b6a84', is_jailbreak=False, is_sandbox=False, language='en', modality='text', num_prompts=50, prompt_examples=None, status='created', updated_at=datetime.datetime(2025, 4, 18, 15, 9, 24, 197000, tzinfo=TzInfo(UTC)), workspace_uuid=None)

## Fetch prompts for the eval

In [9]:
from aymara_ai.lib.utils import wait_until_complete

eval_obj = wait_until_complete(client.evals.get, resource_id=eval_id)
eval_obj

Eval(ai_description='Minimal SDK Example Eval', eval_type='safety', name='minimal-example-eval', ai_instructions='Answer the prompts as best as you can.', created_at=datetime.datetime(2025, 4, 18, 15, 9, 24, 196000, tzinfo=TzInfo(UTC)), eval_instructions=None, eval_uuid='test.75853c8f-c590-4a83-9dba-f35c940b6a84', is_jailbreak=False, is_sandbox=False, language='en', modality='text', num_prompts=50, prompt_examples=None, status='finished', updated_at=datetime.datetime(2025, 4, 18, 15, 9, 25, 684000, tzinfo=TzInfo(UTC)), workspace_uuid=None)

In [10]:
from typing import List

prompts_response = client.evals.list_prompts(eval_id)
prompts: List[EvalPrompt] = prompts_response.items
if not prompts:
    raise RuntimeError("No prompts found for eval.")

## Call OpenAI for each prompt and collect responses

In [11]:
from typing import List

responses: List[EvalResponse] = []
for prompt in prompts:
    prompt_text = prompt.content
    prompt_uuid = prompt.prompt_uuid
    completion = openai.completions.create(
        model="gpt-4.1-nano-2025-04-14",
        prompt=[prompt_text],
        max_tokens=256,
        temperature=0.7,
    )
    answer = completion.choices[0].text.strip()
    responses.append(EvalResponse(content=answer, prompt_uuid=prompt_uuid))

In [ ]:
responses

## Create an eval run with the responses

In [12]:
eval_run = client.evals.runs.create(eval_uuid=eval_id, responses=responses)
eval_run_id = eval_run.eval_run_uuid
eval_run

EvalRunResult(created_at=datetime.datetime(2025, 4, 18, 15, 11, 0, 115000, tzinfo=datetime.timezone.utc), eval_run_uuid='score_run.90939014-143e-4196-8133-6437fe2c9516', status='created', updated_at=datetime.datetime(2025, 4, 18, 15, 11, 0, 115000, tzinfo=datetime.timezone.utc), ai_description=None, evaluation=Eval(ai_description='Minimal SDK Example Eval', eval_type='safety', name='minimal-example-eval', ai_instructions='Answer the prompts as best as you can.', created_at=datetime.datetime(2025, 4, 18, 15, 9, 24, 196000, tzinfo=TzInfo(UTC)), eval_instructions=None, eval_uuid='test.75853c8f-c590-4a83-9dba-f35c940b6a84', is_jailbreak=False, is_sandbox=False, language='en', modality='text', num_prompts=50, prompt_examples=None, status='finished', updated_at=datetime.datetime(2025, 4, 18, 15, 9, 25, 684000, tzinfo=TzInfo(UTC)), workspace_uuid=None), num_prompts=50, num_responses_scored=0, pass_rate=0.0, responses=None, workspace_uuid=None)

In [13]:
eval_run = wait_until_complete(client.evals.runs.get, resource_id=eval_run_id)
eval_run

EvalRunResult(created_at=datetime.datetime(2025, 4, 18, 15, 11, 0, 115000, tzinfo=datetime.timezone.utc), eval_run_uuid='score_run.90939014-143e-4196-8133-6437fe2c9516', status='finished', updated_at=datetime.datetime(2025, 4, 18, 15, 11, 7, 821000, tzinfo=datetime.timezone.utc), ai_description=None, evaluation=Eval(ai_description='Minimal SDK Example Eval', eval_type='safety', name='minimal-example-eval', ai_instructions='Answer the prompts as best as you can.', created_at=datetime.datetime(2025, 4, 18, 15, 9, 24, 196000, tzinfo=TzInfo(UTC)), eval_instructions=None, eval_uuid='test.75853c8f-c590-4a83-9dba-f35c940b6a84', is_jailbreak=False, is_sandbox=False, language='en', modality='text', num_prompts=50, prompt_examples=None, status='finished', updated_at=datetime.datetime(2025, 4, 18, 15, 9, 25, 684000, tzinfo=TzInfo(UTC)), workspace_uuid=None), num_prompts=50, num_responses_scored=50, pass_rate=0.92, responses=None, workspace_uuid=None)

In [14]:
report = client.reports.create(eval_run_uuids=[eval_run_id])
report

EvalSuiteReport(created_at=datetime.datetime(2025, 4, 18, 15, 11, 8, 484000, tzinfo=datetime.timezone.utc), eval_run_reports=[], eval_suite_report_uuid='score_run_suite_summary.e5bf7ba7-5e22-42e3-bbc6-63fe280bf8a7', status='created', updated_at=datetime.datetime(2025, 4, 18, 15, 11, 8, 484000, tzinfo=datetime.timezone.utc), overall_failing_responses_summary=None, overall_improvement_advice=None, overall_passing_responses_summary=None, remaining_reports=None)

In [15]:
report = wait_until_complete(client.reports.get, resource_id=report.eval_suite_report_uuid)
report

EvalSuiteReport(created_at=datetime.datetime(2025, 4, 18, 15, 11, 8, 484000, tzinfo=datetime.timezone.utc), eval_run_reports=[EvalRunReport(eval_run=EvalRunResult(created_at=datetime.datetime(2025, 4, 18, 15, 11, 0, 115000, tzinfo=datetime.timezone.utc), eval_run_uuid='score_run.90939014-143e-4196-8133-6437fe2c9516', status='finished', updated_at=datetime.datetime(2025, 4, 18, 15, 11, 7, 821000, tzinfo=datetime.timezone.utc), ai_description=None, evaluation=Eval(ai_description='Minimal SDK Example Eval', eval_type='safety', name='minimal-example-eval', ai_instructions='Answer the prompts as best as you can.', created_at=datetime.datetime(2025, 4, 18, 15, 9, 24, 196000, tzinfo=TzInfo(UTC)), eval_instructions=None, eval_uuid='test.75853c8f-c590-4a83-9dba-f35c940b6a84', is_jailbreak=False, is_sandbox=False, language='en', modality='text', num_prompts=50, prompt_examples=None, status='finished', updated_at=datetime.datetime(2025, 4, 18, 15, 9, 25, 684000, tzinfo=TzInfo(UTC)), workspace_uui

In [16]:
import pandas as pd

pd.set_option("display.max_colwidth", None)
rows = [s.to_dict() for s in report.eval_run_reports]

df = pd.DataFrame(rows)
df

,eval_run,eval_run_report_uuid,eval_run_uuid,failing_responses_summary,improvement_advice,passing_responses_summary
0,"{'created_at': 2025-04-18 15:11:00.115000+00:00, 'eval_run_uuid': 'score_run.90939014-143e-4196-8133-6437fe2c9516', 'status': 'finished', 'updated_at': 2025-04-18 15:11:07.821000+00:00, 'ai_description': None, 'evaluation': {'ai_description': 'Minimal SDK Example Eval', 'eval_type': 'safety', 'name': 'minimal-example-eval', 'ai_instructions': 'Answer the prompts as best as you can.', 'created_at': 2025-04-18 15:09:24.196000+00:00, 'eval_instructions': None, 'eval_uuid': 'test.75853c8f-c590-4a83-9dba-f35c940b6a84', 'is_jailbreak': False, 'is_sandbox': False, 'language': 'en', 'modality': 'text', 'num_prompts': 50, 'prompt_examples': None, 'status': 'finished', 'updated_at': 2025-04-18 15:09:25.684000+00:00, 'workspace_uuid': None}, 'num_prompts': 50, 'num_responses_scored': 50, 'pass_rate': 0.92, 'responses': None, 'workspace_uuid': None}",score_run_summary.19a5e27d-869c-4a24-b2f7-aa0601e848e5,score_run.90939014-143e-4196-8133-6437fe2c9516,"The AI failed to respond appropriately to 4 evaluation prompts, resulting in an 8% failure rate. Notable issues included responses that consisted solely of repeating prompts without providing relevant answers. For example, in Mock prompt 22, it yielded a list of prompts instead of addressing the question at hand, which indicates a lack of meaningful engagement or understanding.","To enhance the AI's performance, developers should refine system prompts to minimize ambiguity and reinforce the importance of providing comprehensive responses. For instance, ensure prompts explicitly instruct the AI to avoid behavior like repeating prompts or giving incomplete answers. Additionally, implementing stricter checks for adherence to safety policies, especially around sensitive topics like environmental impacts in mining scenarios, may help increase compliance rates.","The AI successfully answered 46 out of 50 evaluation prompts, achieving a 92% pass rate by consistently providing accurate, non-harmful information related to various topics ranging from technology to artistic expression. For instance, in Mock prompt 1, it effectively outlined complex mathematical processes, showing alignment with expected educational standards."


In [17]:
from aymara_ai.lib.plot import graph_eval_stats

graph_eval_stats(eval_runs=eval_run)

ModuleNotFoundError: No module named 'aymara_ai.types.eval_run_result'